In [1]:
#get the probabilities

In [2]:
import pandas as pd
import os
import re
from nltk.stem.porter import *

In [3]:
data = pd.read_csv("Review_Subset.csv")

In [4]:
def getNgrams(text, n):
    if n == 1:
        return text
    
    text.append("<end>")
    ngramList = []
    for i in range(len(text) - (n-1)):
        ngram = text[i:i+n]
        ngramList.append(ngram)
    
    return ngramList

In [5]:
unigramDict = {}
bigramDict = {}
stemmer = PorterStemmer()

def getCountsinRow(row):
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
    unigram = getNgrams(parsedText, 1)
    bigram = getNgrams(parsedText, 2)
    ngrams = (unigram,bigram)
    for index in range(len(ngrams)):
        for eachWord in ngrams[index]:
            
            if index == 0:
                #unigram
                
                if eachWord in unigramDict:
                    unigramDict[eachWord] +=1
                else:
                    unigramDict[eachWord] =1
                    
            if index == 1:
                #bigram
                    eachWord = tuple(eachWord)
                    if eachWord in bigramDict:
                        bigramDict[eachWord] +=1
                    else:
                        bigramDict[eachWord] =1

            

In [6]:

#get the probabilies into unigramsDict and bigramsDict
data.apply(getCountsinRow, axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1970    None
1971    None
1972    None
1973    None
1974    None
1975    None
1976    None
1977    None
1978    None
1979    None
1980    None
1981    None
1982    None
1983    None
1984    None
1985    None
1986    None
1987    None
1988    None
1989    None
1990    None
1991    None
1992    None
1993    None
1994    None
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

In [7]:
print(len(unigramDict))
print(len(bigramDict))

6120
60028


In [8]:
#export unigrams
extractList = []
for eachKey in unigramDict:
    extractList.append([eachKey,unigramDict[eachKey]])

pd.DataFrame(extractList, columns = ["word", "count"]).to_csv("unigramProbabilities.csv", index = False)

In [9]:
#export bigrams
extractList = []
for eachKey in bigramDict:
    extractList.append([eachKey[0],eachKey[1],bigramDict[eachKey]])
    
pd.DataFrame(extractList, columns = ["first", "second", "count"]).to_csv("bigramProbabilities", index = False)

In [10]:
print(data)

      score                                               text
0         5  I have bought several of the Vitality canned d...
1         1  Product arrived labeled as Jumbo Salted Peanut...
2         4  This is a confection that has been around a fe...
3         2  If you are looking for the secret ingredient i...
4         5  Great taffy at a great price.  There was a wid...
5         4  I got a wild hair for taffy and ordered this f...
6         5  This saltwater taffy had great flavors and was...
7         5  This taffy is so good.  It is very soft and ch...
8         5  Right now I'm mostly just sprouting this so my...
9         5  This is a very healthy dog food. Good for thei...
10        5  I don't know if it's the cactus or the tequila...
11        5  One of my boys needed to lose some weight and ...
12        1  My cats have been happily eating Felidae Plati...
13        4  good flavor! these came securely packed... the...
14        5  The Strawberry Twizzlers are my guilty ple

In [11]:
#get the vector model in data
unigramVM = []
def getUnigramReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        score = unigramDict[parsedText[index]]
        rowVector[parsedText[index]] = score
        
    print(rowVector)
    
        
    unigramVM.append(rowVector)
    

data.apply(getUnigramReviewVectors, axis = 1)
    
    
    

{'i': 5434, 'have': 1180, 'bought': 181, 'sever': 64, 'of': 2588, 'the': 6255, 'vital': 1, 'can': 563, 'dog': 216, 'food': 505, 'product': 629, 'and': 4415, 'found': 177, 'them': 903, 'all': 592, 'to': 3119, 'be': 688, 'good': 736, 'qualiti': 128, 'look': 211, 'more': 397, 'like': 937, 'a': 3865, 'stew': 5, 'than': 411, 'process': 32, 'meat': 49, 'it': 2984, 'smell': 55, 'better': 251, 'my': 1366, 'labrador': 1, 'is': 2258, 'finicki': 7, 'she': 151, 'appreci': 10, 'thi': 1913, 'most': 144}
{'product': 629, 'arriv': 82, 'label': 44, 'as': 924, 'jumbo': 2, 'salt': 274, 'peanut': 60, 'the': 6255, 'were': 422, 'actual': 102, 'small': 112, 'size': 143, 'unsalt': 10, 'not': 1103, 'sure': 106, 'if': 539, 'thi': 1913, 'wa': 964, 'an': 279, 'error': 2, 'or': 569, 'vendor': 12, 'intend': 3, 'to': 3119, 'repres': 2}
{'thi': 1913, 'is': 2258, 'a': 3865, 'confect': 2, 'that': 1254, 'ha': 371, 'been': 208, 'around': 70, 'few': 109, 'centuri': 1, 'it': 2984, 'light': 55, 'pillowi': 1, 'citru': 6, 'ge

{'i': 5434, 'm': 241, 'present': 14, 'on': 875, 'a': 3865, 'diet': 72, 'and': 4415, 'wa': 964, 'at': 590, 'my': 1366, 'fresh': 134, 'easi': 115, 'neighborhood': 4, 'groceri': 91, 'look': 211, 'over': 151, 'possibl': 21, 'food': 505, 'want': 198, 'thing': 166, 'that': 1254, 'were': 422, 'tasti': 131, 'non': 35, 'fat': 154, 'low': 108, 'in': 1796, 'calori': 176, 'came': 85, 'home': 65, 'with': 1126, 'about': 350, 'dozen': 6, 'item': 98, 's': 937, 'how': 149, 'discov': 31, 'the': 6255, 'tillen': 2, 'farm': 39, 'pickl': 8, 'crispi': 31, 'asparagu': 11, 'well': 208, 've': 253, 'alway': 99, 'like': 937, 'anyway': 25, 'enjoy': 165, 'sever': 64, 'brand': 266, 'of': 2588, 'thi': 1913, 'is': 2258, 'realli': 340, 'good': 736, 'it': 2984, 'best': 297, 'can': 563, 'recal': 5, 'there': 294, 'an': 279, 'excel': 93, 'flavor': 982, 'big': 90, 'clove': 3, 'garlic': 50, 'down': 79, 'bottom': 43, 'jar': 51, 'which': 269, 'forward': 12, 'to': 3119, 'eat': 386, 't': 946, 'believ': 54, 'tast': 820, 'right': 

{'thi': 1913, 'packet': 37, 'of': 2588, 'glaze': 11, 'is': 2258, 'the': 6255, 'secret': 6, 'to': 3119, 'make': 380, 'those': 105, 'european': 7, 'style': 26, 'fresh': 134, 'fruit': 67, 'tart': 21, 'i': 5434, 'am': 262, 'about': 350, 'one': 590, 'for': 1824, 'a': 3865, 'pie': 6, 'auction': 2, 'at': 590, 'church': 1, 'after': 207, 'friend': 90, 'request': 3, 'that': 1254, 'someth': 112, 'strawberri': 32, 'should': 61, 'be': 688, 'offer': 35, 'you': 1273, 'can': 563, 'most': 144, 'amaz': 62, 'and': 4415, 'rel': 7, 'healthi': 120, 'by': 258, 'bake': 119, 'base': 35, 'layer': 9, 'it': 2984, 'with': 1126, 'uncook': 2, 'slice': 21, 'then': 167, 'pour': 17, 'over': 151, 'made': 148, 'up': 268, 'content': 31, 'these': 1197, 'they': 1331, 'harden': 2, 'room': 10, 'temperatur': 7, 'hold': 24, 'togeth': 28, 'so': 845, 'let': 48, 'beauti': 17, 'arrang': 1, 'show': 19, 'through': 82, 've': 253, 'also': 291, 'seen': 26, 'use': 542, 'cheesecak': 6, 'top': 62, 'in': 1796, 'place': 42, 'person': 51, 'do

{'the': 6255, 'recommend': 197, 'when': 390, 'we': 523, 'bought': 181, 'our': 199, 'puppi': 18, 'lhasa': 2, 'bichon': 3, 'mix': 178, 'and': 4415, 'poodl': 4, 'wa': 964, 'that': 1254, 'continu': 31, 'them': 903, 'on': 875, 'food': 505, 'they': 1331, 'have': 1180, 'had': 445, 'sinc': 148, 'be': 688, 'wean': 4, 'thi': 1913, 'is': 2258, 'a': 3865, 'somewhat': 23, 'expens': 73, 'so': 845, 'i': 5434, 'can': 563, 't': 946, 'say': 173, 'were': 422, 'terribl': 25, 'pleas': 68, 'but': 1178, 'load': 13, 'up': 268, 'stuff': 92, 'head': 8, 'home': 65, 'br': 2137, 'let': 48, 'me': 375, 'just': 543, 'small': 112, 'dog': 216, 'befor': 131, 'never': 122, 'seen': 26, 'such': 60, 'ga': 14, 'issu': 49, 'both': 88, 'of': 2588, 'consist': 44, 'foul': 2, 'usual': 73, 're': 181, 'sit': 19, 'lap': 2, 'you': 1273, 'know': 168, 'got': 126, 'through': 82, 'first': 184, 'bag': 630, 'purchas': 208, 'more': 397, 'amazon': 366, 'it': 2984, 'great': 633, 'price': 303, 'for': 1824, '40': 12, 'lb': 17, 'think': 167, '15

{'i': 5434, 'love': 671, 'thi': 1913, 'coffe': 236, 'do': 268, 'wish': 64, 'it': 2984, 'came': 85, 'alreadi': 23, 'ground': 7, 'as': 924, 'don': 316, 't': 946, 'have': 1180, 'a': 3865, 'grinder': 2, 'and': 4415, 'use': 542, 'blender': 3, 'instead': 63, 'but': 1178, 's': 937, 'worth': 60, 'mani': 160, 'flavor': 982, 've': 253, 'tri': 567, 'onli': 345, 'seem': 116, 'to': 3119, 'odor': 5, 'that': 1254, 'doesn': 71, 'reach': 4, 'the': 6255, 'one': 590, 'manag': 7, 'both': 88}
{'thi': 1913, 'brand': 266, 'of': 2588, 'coffe': 236, 'ha': 371, 'been': 208, 'avail': 60, 'in': 1796, 'my': 1366, 'local': 128, 'store': 251, 'for': 1824, 'some': 326, 'time': 326, 'i': 5434, 've': 253, 'believ': 54, 'the': 6255, 'compani': 73, 'is': 2258, 'drink': 145, 'it': 2984, 'quit': 60, 'and': 4415, 'have': 1180, 'not': 1103, 'tire': 19, 'm': 241, 'particularli': 13, 'fond': 6, 'turtl': 2, 'sunda': 2, 'but': 1178, 'all': 592, 'flavor': 982, 'are': 1504, 'good': 736, 'can': 563, 't': 946, 'say': 173, 'that': 12

...

{'i': 5434, 'tri': 567, 'the': 6255, 'martha': 4, 'white': 64, 'turtl': 2, 'fudg': 3, 'chocol': 135, 'chip': 1184, 'cooki': 65, 'recip': 49, 'and': 4415, 'these': 1197, 'are': 1504, 'worst': 17, 'have': 1180, 'ever': 130, 'made': 148, 'either': 54, 'there': 294, 'is': 2258, 'someon': 28, 'on': 875, 'staff': 4, 'of': 2588, 'who': 144, 'ha': 371, 'a': 3865, 'lopsid': 1, 'sens': 6, 'humor': 1, 'or': 569, 'their': 214, 'tast': 820, 'bud': 9, 'taken': 7, 'perman': 1, 'vacat': 12, 'wast': 38, 'my': 1366, 'time': 326, 'lot': 123, 'money': 71, 'pecan': 4, 'effort': 8, 'need': 133, 'to': 3119, 'tweek': 1, 'thi': 1913, 'take': 118, 'it': 2984, 'off': 123, 'websit': 20, 'so': 845, 'no': 349, 'other': 439, 'unsuspect': 3, 'housewif': 1, 'will': 431, 'her': 116, 'worthless': 2, 'husband': 74, 'pick': 35, 'up': 268, 'one': 590, 'hi': 80, 'way': 151, 'through': 82, 'kitchen': 16, 'within': 22, 'two': 131, 'second': 43, 'he': 190, 'came': 85, 'back': 86, 'in': 1796, 'threw': 14, 'garbag': 11}
{'i': 54

{'i': 5434, 'wa': 964, 'on': 875, 'the': 6255, 'fenc': 3, 'about': 350, 'buy': 374, 'these': 1197, 'for': 1824, 'month': 107, 'read': 56, 'review': 156, 'and': 4415, 'were': 422, 'impress': 25, 'by': 258, 'them': 903, 'but': 1178, 'possibl': 21, 'of': 2588, 'be': 688, 'stuck': 17, 'with': 1126, '24': 43, 'bag': 630, 'chip': 1184, 'hate': 33, 'made': 148, 'me': 375, 'nervou': 4, 'thi': 1913, 'my': 1366, 'first': 184, 'food': 505, 'order': 388, 'from': 457, 'amazon': 366, 'am': 262, 'glad': 45, 'pick': 35, 'product': 629, 'to': 3119, 'absolut': 57, 'love': 671, 'deic': 1, 'varieti': 146, 'pack': 249, 'start': 76, 'in': 1796, 'case': 119, 'didn': 108, 't': 946, 'like': 937, 'one': 590, 'flavor': 982, 'had': 445, 'other': 439, 'could': 155, 'eat': 386, 'turn': 31, 'out': 360, 'though': 97, 'enjoy': 165, 'all': 592, 'br': 2137, 'they': 1331, 'are': 1504, 'thicker': 12, 'than': 411, 'your': 259, 'normal': 48, 'potato': 302, 'a': 3865, 'great': 633, 'crunch': 57, 'more': 397, 'an': 279, 'airi

{'overal': 29, 'the': 6255, 'chip': 1184, 'were': 422, 'veri': 577, 'tastey': 4, 'my': 1366, 'pickey': 2, 'daughter': 49, 'even': 230, 'like': 937, 'then': 167, 'so': 845, 'that': 1254, 'is': 2258, 'a': 3865, 'real': 74, 'plu': 55, 'for': 1824, 'me': 375, 'some': 326, 'flavor': 982, 'better': 251, 'but': 1178, 'all': 592, 'did': 134, 'meet': 11, 'expect': 84, 'tri': 567, 'it': 2984, 'and': 4415, 'see': 80, 'yourelf': 1, 'enjoy': 165}
{'thi': 1913, 'chip': 1184, 'are': 1504, 'realli': 340, 'good': 736, 'and': 4415, 'it': 2984, 's': 937, 'fun': 27, 'get': 384, 'the': 6255, 'varieti': 146, 'pack': 249, 'now': 212, 'they': 1331, 'as': 924, 'regular': 132, 'probabl': 68, 'not': 1103, 'but': 1178, 're': 181, 'also': 291, 'nearli': 16, 'fatten': 6, 'bad': 118, 'for': 1824, 'you': 1273, 'given': 32, 'that': 1254, 'these': 1197, 'a': 3865, 'healthier': 58, 'version': 37, 'if': 539, 'want': 198, 'i': 5434, 'would': 353, 'highli': 83, 'recommend': 197, 'them': 903}
{'i': 5434, 'order': 388, 'thi'

{'thought': 89, 'i': 5434, 'wa': 964, 'order': 388, 'summer': 23, 'tea': 371, 'flavor': 982, 'got': 126, 'leftov': 7, 'winter': 6, 'garbag': 11, 'will': 431, 'not': 1103, 'buy': 374, 'from': 457, 'thi': 1913, 'seller': 23, 'again': 211, 'hate': 33, 'ice': 72, 'coffe': 236, 'much': 317, 'less': 111, 'gingerbread': 13, 'and': 4415, 'eggnog': 5, 'cannot': 33, 'believ': 54, 'just': 543, 'spent': 5, 'that': 1254, 'amount': 81, 'of': 2588, 'money': 71, 'on': 875, 'crap': 4, 'ever': 130, 'drink': 145, 'should': 61, 'have': 1180, 'wait': 21, 'until': 47, 'walmart': 13, 'carri': 47, 'it': 2984, 'doe': 100, 'keurig': 12, 'know': 168, 'stuff': 92, 'is': 2258, 'be': 688, 'sold': 24, 'under': 24, 'such': 60, 'fals': 1, 'advertis': 23, 'ye': 28, 'say': 173, 'may': 77, 'substitut': 28, 'would': 353, 'fine': 38, 'if': 539, 'they': 1331, 'were': 422, 'more': 397, 'had': 445, 'want': 198, 'gross': 11, 'br': 2137, 'the': 6255, 'up': 268, 'side': 63, 'peach': 5, 'receiv': 82, 'veri': 577, 'good': 736, 'ot

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1970    None
1971    None
1972    None
1973    None
1974    None
1975    None
1976    None
1977    None
1978    None
1979    None
1980    None
1981    None
1982    None
1983    None
1984    None
1985    None
1986    None
1987    None
1988    None
1989    None
1990    None
1991    None
1992    None
1993    None
1994    None
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

In [12]:
pd.DataFrame(unigramVM).to_csv("UnigramVectorModel.csv", index = False)

In [13]:
len(unigramVM)
len(unigramDict)

6120